# Import Python module

In [1]:
import matplotlib.pyplot as plt
import time

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont

In [3]:
import numpy as np 
import cv2
from pywt import dwt,idwt
import os

#### 임시
***

In [4]:
# phone = "galaxyS7"
p = "phone3"
img_num = 2

***

## 1. Attack

In [5]:
att = []
ext = []

In [6]:
emd_url = 'embed/captured_{}_img{}.png'.format(p,img_num)

#### jpg compression

In [7]:
att_url = 'attack/png_to_jpg/p2j_{}_img{}.jpg'.format(p,img_num)
att.append(att_url)
ext.append('extract/attack/png_to_jpg/wm_p2j_{}_img{}.png'.format(p,img_num))

im = Image.open(emd_url)
rgb_im = im.convert('RGB')
rgb_im.save(att_url)

#### median filtering

In [8]:
att_url = 'attack/median/m_{}_img{}.png'.format(p,img_num)
att.append(att_url)
ext.append('extract/attack/median/wm_m_{}_img{}.png'.format(p,img_num))

img = cv2.imread(emd_url)
m_blur = cv2.medianBlur(img,3)
cv2.imwrite(att_url,m_blur)

True

#### Gaussian filtering

In [9]:
att_url = 'attack/gaussian/g_{}_img{}.png'.format(p,img_num)
att.append(att_url)
ext.append('extract/attack/gaussian/wm_g_{}_img{}.png'.format(p,img_num))

img = cv2.imread(emd_url)
g_blur = cv2.GaussianBlur(img,(3,3),0)
cv2.imwrite(att_url,g_blur)

True

#### Average filtering

In [10]:
att_url = 'attack/average/a_{}_img{}.png'.format(p,img_num)
att.append(att_url)
ext.append('extract/attack/average/wm_a_{}_img{}.png'.format(p,img_num))

img = cv2.imread(emd_url)
a_blur = cv2.blur(img,(3,3))
cv2.imwrite(att_url,a_blur)

True

#### Histogram Equalization(X)

In [11]:
# att_url = 'code_fix/embed/hist_equal_code3.png'

# img = cv2.imread(emd_url)
# img2 = []
# for i in range(3):
#     img2.append(cv2.equalizeHist(img[:,:,i]))
    
# img_merge = cv2.merge(img2)
# cv2.imwrite(att_url,img_merge)

#### salt and pepper(X)

In [12]:
# att_url = 'code_fix/embed/s&p_dwt1_88_two_0.005.png'

# img = cv2.imread(emd_url)
# from random import random

# def salt_and_pepper(image, p): 
#     output = np.zeros(image.shape,np.uint8) 
#     thres = 1 - p 
#     for i in range(image.shape[0]): 
#         for j in range(image.shape[1]): 
#             rdn = random() 
#             if rdn < p: 
#                 output[i][j] = 0 
#             elif rdn > thres: 
#                 output[i][j] = 255 
#             else: 
#                 output[i][j] = image[i][j]
#     return output

# salt_pepper_noise_imgs = salt_and_pepper(img,0.005)
# cv2.imwrite(att_url,salt_pepper_noise_imgs)

### 2. extract

제약1) wm_shape 알아야 함(13,148 통일)

In [13]:
def block_get_wm(block):
    
    mod = 32
    
    block_dct = cv2.dct(block)
    U,s,V = np.linalg.svd(block_dct)
    max_s = s[0]
    
    if max_s%mod > mod/2:
        wm_1 = 255
    else:
        wm_1= 0
    
    return wm_1

In [14]:
for k in range(len(att)):
    print(k)
    emd_url = att[k]
    embed_img = cv2.imread(emd_url).astype(np.float32)
    

    # font10
    wm_shape = (13,148)

    dwt_deep = 1
    block_shape = (8,8)

    embed_img_YUV = cv2.cvtColor(embed_img, cv2.COLOR_BGR2YUV)


    if not embed_img_YUV.shape[0]%(2**dwt_deep)==0:
        temp = (2**dwt_deep)-embed_img_YUV.shape[0]%(2**dwt_deep)
        embed_img_YUV = np.concatenate((embed_img_YUV,np.zeros((temp,embed_img_YUV.shape[1],3))),axis=0)
    if not embed_img_YUV.shape[1]%(2**dwt_deep)==0:
        temp = (2**dwt_deep)-embed_img_YUV.shape[1]%(2**dwt_deep)
        embed_img_YUV = np.concatenate((embed_img_YUV,np.zeros((embed_img_YUV.shape[0],temp,3))),axis=1)

    # assert embed_img_YUV.shape[0]%(2**dwt_deep)==0
    # assert embed_img_YUV.shape[1]%(2**dwt_deep)==0

    coeffs_Y = dwt(embed_img_YUV[:,:,0],'haar')
    ha_Y = coeffs_Y[0]
    coeffs_Y = coeffs_Y[1]

    shape0_int,shape1_int = ha_Y.shape[0]//block_shape[0], ha_Y.shape[1]//block_shape[1]
    block_index0,block_index1 = np.meshgrid(np.arange(shape0_int),np.arange(shape1_int))
    b_flat_idx0, b_flat_idx1 = block_index0.flatten(),block_index1.flatten()
    length = b_flat_idx0.size

    ha_block_shape = (int(ha_Y.shape[0]/block_shape[0]),int(ha_Y.shape[1]/block_shape[1]),block_shape[0],block_shape[1])
    strides = ha_Y.itemsize*(np.array([ha_Y.shape[1]*block_shape[0],block_shape[1],ha_Y.shape[1],1]))

    ha_Y_block = np.lib.stride_tricks.as_strided(ha_Y.copy(),ha_block_shape,strides)
    # coeffs_Y_block = np.lib.stride_tricks.as_strided(coeffs_Y.copy(),ha_block_shape,strides)

    extract_wm = np.array([])


    for i in range(length):

        wm = block_get_wm(ha_Y_block[b_flat_idx0[i],b_flat_idx1[i]])

    #     wm2 = block_get_wm(coeffs_Y_block[b_flat_idx0[i],b_flat_idx1[i]])

        # i가 워터마크의 크기를 벗어난 경우
        if i<wm_shape[0]*wm_shape[1]:
            extract_wm  = np.append(extract_wm,wm)

    #         extract_wm2 = np.append(extract_wm2,wm2)

        else:
            times = i//(wm_shape[0]*wm_shape[1])
            ii = i%(wm_shape[0]*wm_shape[1])
            extract_wm[ii] = (extract_wm[ii]*times + wm)/(times+1)

    #         extract_wm2[ii] = (extract_wm2[ii]*times + wm2)/(times+1)
    
   

    cv2.imwrite(ext[k],extract_wm.reshape(wm_shape[0],wm_shape[1]))

0
1
2
3
